In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sys
import torchvision
from torchvision.datasets import CIFAR10
from models.network_util import init_net
import torchvision.transforms as transforms
# import configs.base_config

In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
sys.path.append('../')

In [8]:
from models.resnet_cifar import Resnet50

model = Resnet50()

In [9]:
transformation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop((32, 32), 4),
    transforms.ToTensor()
])
dataset = CIFAR10('../../dataset/', train=True, transform=transformation, download=True)

Files already downloaded and verified


In [10]:
dataloader = DataLoader(dataset, batch_size=)

In [12]:
init_net(model)

initialize network with normal


Resnet50(
  (first_layer): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (stages): Sequential(
    (0): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (build_block): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BuildingBlock(
      (relu): ReLU()
      (residual): Sequential(
        (0): Conv2d(16, 16, k

In [26]:
for i, (img, label) in enumerate(dataloader):
    print(img.size())
    if i > 3:
        break

torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
